In [66]:
import pickle
import datetime
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R
import seaborn as sns
from pympler import asizeof
from scipy import interpolate
from ahrs.filters import EKF
from ahrs import QuaternionArray
#%matplotlib inline
%matplotlib widget

In [123]:
folder = 'Data'
user = 15
with open(folder + '/P' + str(user) + '_Processed.pickle', 'rb') as handle:
    session_data, entrance, exit = pickle.load(handle)

In [124]:
def printSurveys(session):
    surveys = session['surveys']
    for s in surveys:
        print(s)
        for k in surveys[s]:
            print(f'\t{k}[{type(surveys[s][k])}]: {surveys[s][k]}') 

            
def session_times(session):
    ''' given a session, send back ((start_time, end_time), (transition_start, noticed), unnoticed_timedelta, extra_timedelta, duration_timedelta) '''
    #session started
    start_time = session['timings'][session['timings']['event']=='START_TEST'].index.values[0]
    
    #additional time after noticing the light but before starting the survey.  For the second phase of each activity there was an option to continue playing tetris/flow activity
    extra_time = session['timings'][session['timings']['event']=='START_SURVEY'].index.values[-1]-session['timings'][session['timings']['event']=='NOTICED'].index.values[0]
    
    #timing of the start of the LED change
    transition_start = session['timings'][session['timings']['event']=='START_TRANSITION'].index.values[0]
    #time of noticing
    noticed = session['timings'][session['timings']['event']=='NOTICED'].index.values[-1]
    
    #session ends when noticed...
    end_time = noticed
    #...unless they actually took extra time (i.e. more than 30 sec). then set the end_time to when they start the survey instead.
    if datetime.timedelta(seconds=extra_time.astype('timedelta64[s]') / np.timedelta64(1, 's')) > datetime.timedelta(seconds=30):
        end_time = session['timings'][session['timings']['event']=='START_SURVEY'].index.values[-1]
        
    #calc duration and unnoticed time
    duration = end_time - start_time 
    unnoticed = noticed - transition_start
    
    return (start_time, end_time), (transition_start, noticed), unnoticed.astype("timedelta64[s]"), extra_time.astype("timedelta64[s]"), duration.astype("timedelta64[s]")


def chop_df(df, taskbounds):
    """
    This function filters a dataframe based on the provided start and end times.
    The dataframe should have timestamps as the index.
    
    Args:
        df (pd.DataFrame): Input dataframe with timestamps as the index.
        taskbounds: Tuple of (start_time, end_time), where each is an (np.datetime64)
        
    Returns:
        pd.DataFrame: A new dataframe with only the data between the provided start and end times.
    """
    start_time, end_time = taskbounds

    start_time = pd.to_datetime(start_time).tz_localize('UTC')
    end_time = pd.to_datetime(end_time).tz_localize('UTC')

    # Filter the dataframe
    df_filtered = df[(df.index >= start_time) & (df.index <= end_time)]

    new_length = len(df_filtered)
    chopped_beginning = df[df.index < start_time].shape[0]
    chopped_end = df[df.index > end_time].shape[0]

    # Calculate and print the number of samples chopped off the beginning and end of the dataframe
    print(f"Filtered dataframe length: {new_length}")
    print(f"Number of samples chopped off from the beginning: {chopped_beginning}")
    print(f"Number of samples chopped off from the end: {chopped_end}")

    return df_filtered
   

def pull_activity_data(session, df_name, timebounds=None):
    ''' given a session, return the data that falls between the start and end times of the task (started the task to the notice event, or the survey if they took extra time) if timebounds==None.
    if timebounds are passed, pull the data in between those timebounds.  df_name can take on values 'blinks','acc','gyro','quaternions','thermal','watch_temp','watch_lux' '''
    
    if timebounds == None:
        timebounds, _, _, _, _ = session_times(session)
    
    return chop_df(session[df_name], taskbounds)

In [ ]:
PROCESS SURVEYS



In [127]:
for session in session_data:
    if session['session_type'] in ['LAB_TETRIS', 'TETRIS_CONTD', 'LAB_FLOW', 'FLOW_CONTD', 'HOME_TETRIS', 'HOME_FLOW']:
        
        print(session['session_type'])
        taskbounds, transitionbounds, unnoticed_dur, extra_dur, task_dur = session_times(session)
        
        print(f'UNNOTICED TIME: {unnoticed_dur}')
        print(f'    EXTRA TIME: {extra_dur}')
        print(f'      DURATION: {task_dur}')
    
        for k in session['surveys']:
            survey = session['surveys'][k]
            if (k == 'LabStartSurvey'):
                print('IGNORE Lab Start Survey (had people do it online')
                print('May need to move results from P1-3 to online CSV')
        
            else:
                print(survey)

LAB_TETRIS
UNNOTICED TIME: 54 seconds
    EXTRA TIME: 17 seconds
      DURATION: 780 seconds
{'nowAlertness': 5, 'nowStress': 5, 'nowEmotion': 3, 'nowEmoIntensity': 2, 'freeEmotion': ['Focused', ' mildly annoyed at controls', ' feel adrenaline pumping'], 'freeFood': 'Coffee a few hours ago', 'freeAdditional': '', 'reactionTimesMs': [376, 1259, 324, 323, 509, 323, 341, 410, 326, 381, 323, 323, 324], 'LastSeenTimeStartSurvey': datetime.datetime(2023, 4, 14, 14, 14, 48)}
{'duration': 7, 'actualTimeAtDuration': datetime.datetime(2023, 4, 14, 14, 28, 5)}
TETRIS_CONTD
UNNOTICED TIME: 19 seconds
    EXTRA TIME: 280 seconds
      DURATION: 993 seconds
{'startSurveyTime': datetime.datetime(2023, 4, 14, 14, 40, 10), 'durGuess': 25, 'timeAtDurGuess': datetime.datetime(2023, 4, 14, 14, 41, 14), 'durTimeConfidence': 2, 'guessTime': datetime.datetime(2023, 4, 14, 14, 45), 'timeAtGuess': datetime.datetime(2023, 4, 14, 14, 40, 35), 'guessTimeConfidence': 2, 'flowPath': '', 'flowCanvasSize': ['null', '